# Libraries

In [1]:
#pip install underthesea

In [1]:
import underthesea
underthesea.__version__

'1.1.5'

In [2]:
import math
from collections import Counter

from underthesea import word_sent #word_tokenize
from underthesea import pos_tag

# I. Xây dựng chương trình gán nhãn từ loại cho tiếng Việt

## a. Xây dựng chương trình

### Xử lý dữ liệu huấn luyện

In [3]:
filename = 'vi_train.pos'
with open(filename, encoding='utf8') as f:
    lines = f.readlines()

print(len(lines))

16986


In [4]:
train_data = []
list_word = []
for line in lines:
    words = line.split()
    sent = []
    ignore = False
    for w in words:
        tmp = w.split('/')

        if ( len(tmp)!=2 ):
            ignore = True
            break
        sent.append((tmp[0], tmp[1].upper()))
        list_word.append(tmp[0])
    if ignore:
        continue
    train_data.append(sent)

list_word = set(list_word) #bỏ những từ bị trùng
list_word = list(list_word)

train_data = [t for t in train_data if t!=[]] #bỏ các dòng rỗng []
print(len(train_data))

8303


In [5]:
def get_ngram(sent):
    tag_gram = []
    tagtag_gram = []
    tagword_gram = []
    for i in range(len(sent)):
        if i==0:
            tag_gram.append('T0')
            tagtag = 'T0' + '_' + sent[i][1]
        else:
            tagtag = sent[i-1][1] + '_' + sent[i][1]
        tag_gram.append(sent[i][1])
        tagword = sent[i][1] + '_' + sent[i][0] 
        tagtag_gram.append(tagtag)
        tagword_gram.append(tagword)
    return tag_gram, tagtag_gram, tagword_gram

In [6]:
tag_gram_all = []
tagtag_gram_all = []
tagword_gram_all = []

for i in range(len(train_data)):
    tag_gram, tagtag_gram, tagword_gram = get_ngram(train_data[i])
    tag_gram_all.extend(tag_gram)
    tagtag_gram_all.extend(tagtag_gram)
    tagword_gram_all.extend(tagword_gram)

In [7]:
tag_counter = Counter()
tag_counter.update(tag_gram_all)
print(tag_counter.most_common(20))

print()
tagtag_counter = Counter()
tagtag_counter.update(tagtag_gram_all)
print(tagtag_counter.most_common(20))

print()
tagword_counter = Counter()
tagword_counter.update(tagword_gram_all)
print(tagword_counter.most_common(20))

[('N', 44889), ('V', 37178), ('R', 13931), ('E', 11829), ('A', 11763), (',', 10305), ('T0', 8303), ('P', 8157), ('NP', 7438), ('M', 7031), ('.', 7009), ('C', 6921), ('NC', 4775), ('L', 3434), ('“', 2272), ('”', 2228), ('T', 1255), ('...', 1215), (':', 1205), ('-', 1188)]

[('V_N', 9816), ('N_V', 9538), ('N_N', 8033), ('V_V', 7076), ('R_V', 6916), ('E_N', 5862), ('N_A', 4117), ('V_E', 3500), ('N_,', 3270), ('N_E', 3249), ('M_N', 3244), ('V_R', 3216), ('N_R', 3081), (',_N', 2743), ('NC_N', 2666), ('L_N', 2594), ('P_V', 2573), ('N_NP', 2438), ('V_A', 2404), ('N_P', 2363)]

[(',_,', 10304), ('._.', 7009), ('“_“', 2272), ('”_”', 2226), ('E_của', 1996), ('M_một', 1847), ('C_và', 1702), ('V_có', 1645), ('R_đã', 1553), ('V_là', 1474), ('R_không', 1473), ('L_những', 1439), ('N_người', 1285), ('..._...', 1215), (':_:', 1205), ('-_-', 1188), ('E_trong', 1043), ('R_cũng', 966), ('E_với', 956), ('P_tôi', 950)]


### Tính tham số Model

In [8]:
#Laplace smoothing
anpha = 0.001
V = len(tag_counter)

In [9]:
### P(tj|ti) ->log
def cal_logP_tt(tag1, tag2):
    tag1_tag2 = tag1 + '_' + tag2
    P = (tagtag_counter[tag1_tag2] + anpha)/(tag_counter[tag1] + V*anpha)
    return math.log(P)

logP_tagtag = {}
for t1_t2 in tagtag_counter:
    t1 = t1_t2.split('_')[0]
    t2 = t1_t2.split('_')[1]
    logP = cal_logP_tt(t1, t2)
    logP_tagtag[t1_t2] = logP

In [10]:
### P(wi|tj) ->log
def cal_logP_tw(tag, word):
    tag_word = tag + '_' + word
    P = (tagword_counter[tag_word] + anpha)/(tag_counter[tag] + V*anpha)
    return math.log(P)

logP_tagword = {}
for t_w in tagword_counter:
    t = t_w.split('_')[0]
    w = t_w.split('_')[1]
    logP = cal_logP_tw(t, w)
    logP_tagword[t_w] = logP

### Viterbi Algorithm

In [11]:
list_tag = []
for tag in tag_counter:
    list_tag.append(tag)

#Danh sách tất cả các tag trừ T0
list_tag.remove('T0')
print(list_tag)

['E', 'N', 'V', ',', 'M', 'R', 'NP', '.', 'A', '...', 'P', '-', 'NY', '“', '”', ':', 'L', 'C', 'NC', 'X', '(', ')', 'NU', 'NB', '"', 'T', '!', '&', 'S', ';', 'AP', '?', 'B', '*', '', 'I', '…', 'Y', ']', '>', 'VY', 'VB', '.)', 'AB', '’', '~', 'EB', '):', 'H', '+']


In [12]:
#sent: một câu nhập vào dưới dạng list các từ
#sent = [w1, w2, w3, w4,...]

#paths: lưu chuỗi các nhãn có thể xảy ra tương ứng với các từ trong sent
#paths = [[t0, t1, t2, t1], (i = 1)
#         [t0, t2, t1, t2], (i = 2)
#         [.............]]

#log_values: lưu lại giá trị tốt nhất đã tính trước đó (tương ứng với từng chuỗi các nhãn đã lưu ở paths)
#log_values = [logP1, (i = 1)
#              logP2, (i = 2)
#              ...]

def viterbi_algo(sent):
    paths = []
    log_values = []
    for i in range(len(list_tag)):
        paths.append(['T0'])
        log_values.append(0)

    for word in sent:
        #Tránh làm mất giá trị path và log cũ khi đang xét các tag
        new_path = []
        new_values = []

        #dành cho điều kiện biến temp ở dưới
        #tránh trường hợp list_v rỗng
        num_tagword = 0
        for tag in list_tag:
            x_word = tag + '_' + word
            if x_word in logP_tagword.keys():
                num_tagword += 1

        for i in range(len(list_tag)): #duyệt tag cho từ tiếp theo
            tag_curr = list_tag[i]
            list_v = [] #(paths[j] có giá trị log tương ứng ở list_v[j])

            #Chỉ duyệt các tag có khả năng xảy ra với word
            temp = tag_curr + '_' + word
            if temp not in logP_tagword.keys() and num_tagword!=0:
                continue

            for j in range(len(paths)): #duyệt qua các kết quả cũ
                tag_pre = paths[j][-1]
                tag_tag = tag_pre + '_' + tag_curr
                tag_word = tag_curr + '_' + word

                if tag_tag in logP_tagtag.keys():
                    logP1 = logP_tagtag[tag_tag]
                else:
                    logP1 = cal_logP_tt(tag_pre, tag_curr)

                if tag_word in logP_tagword.keys():
                    logP2 = logP_tagword[tag_word]
                else:
                    logP2 = cal_logP_tw(tag_curr, word)

                #Tính tổng logP(wi|ti) và logP(ti|tj)
                #và cộng thêm giá trị log_values[j] chứa giá trị tốt nhất đã lưu trước đó
                logP = log_values[j] + logP1 + logP2
                #append vao list v
                list_v.append(logP)

            #tìm ra vị trí chứa giá trị tốt nhất hiện tại
            max_index = 0
            for index in range(len(list_v)):
                if (list_v[index] > list_v[max_index]) and (list_v[index]!=0):
                    max_index = index

            #lưu lại giá trị tốt nhất kèm theo tag tương ứng
            p = paths[max_index].copy()
            p.append(tag_curr)
            new_path.append(p)
            new_values.append(list_v[max_index])
        paths = new_path
        log_values = new_values

    index = 0
    for i in range(len(log_values)):
        if (log_values[i]>log_values[index]):
            index = i

    paths[index].remove('T0')
    return paths[index]

## b. Đánh giá độ chính xác trên tập test

In [13]:
filename = 'vi_test.pos'
with open(filename, encoding='utf8') as f:
    lines = f.readlines()

#------------------------------------------#

test_data = []
for line in lines:
    words = line.split()
    sent = []
    ignore = False
    for w in words:
        tmp = w.split('/')
        if ( len(tmp)!=2 ):
            ignore = True
            break
        sent.append((tmp[0], tmp[1].upper()))
    if ignore:
        continue
    test_data.append(sent)

test_data = [t for t in test_data if t!=[]]
print(len(test_data))

1032


In [14]:
y_true = []
#y_true = [tags_sent1, tags_sent2,...]
#tags_sent1 = tag1, tag2,...
x_test = [] 
#x_test = [sentence1, sentence2,...]
#sentence1 = [word1, word2,...]

for sent in test_data:
    s = []
    for data in sent:
        s.append(data[0])
        y_true.append(data[1])
    x_test.append(s)

print(len(test_data))
test_data = [t for t in test_data if t!=[]]

1032


In [15]:
#Gắn nhãn bằng thuật toán Viterbi
y_pred = []
for sent in x_test:
    tag = viterbi_algo(sent)
    y_pred.extend(tag)

In [16]:
print(len(y_true))
print(len(y_pred))

22758
22758


In [17]:
def cal_accurary(true_values, pred_values):
    if ( len(true_values)!=len(pred_values) ):
        return 0
    count = 0
    for i in range(len(true_values)):
        if ( true_values[i]==pred_values[i] ):
            count += 1
    return count/len(true_values)

In [18]:
#Tính độ chính xác nhãn vừa tìm được so với nhãn đúng
print('Accuracy: ', end='')
print(cal_accurary(y_true, y_pred))

Accuracy: 0.8918182617101679


## c. Input vào một câu tiếng Việt, cho ra nhãn của từng từ trong câu

In [19]:
def POS_tagger(sentence):
    word_segment = word_sent(sentence, format="text")
    s = word_segment.split() #cho đúng với format đầu vào hàm viterbi_algo
    paths = viterbi_algo(s)

    result = ''
    for i in range(len(s)):
        result += s[i] + '/' + paths[i] + ' '
    return result

In [20]:
sent1 = 'Dù khá đắt nhưng tôi vẫn đồng ý.'
sent2 = 'Mình không nuôi chúng nó được.'
sent3 = 'Nhà tôi ở trên tầng năm.'

print(POS_tagger(sent1))
print(POS_tagger(sent2))
print(POS_tagger(sent3))

Dù/C khá/R đắt/A nhưng/C tôi/P vẫn/R đồng_ý/V ./. 
Mình/P không/R nuôi/V chúng_nó/P được/R ./. 
Nhà_tôi/* ở/V trên/E tầng/N năm/N ./. 


# II. Sử dụng thư viện CRF (Conditional Random Fields) để giải quyết bài toán

In [21]:
def CRF_POS_tagger(sentence):
    word_segment = word_sent(sentence, format="text")
    list_word_tag = pos_tag(word_segment)
    
    result = ''
    for w_t in list_word_tag:
        result += w_t[0] + '/' + w_t[1] + ' '
    return result

In [22]:
sent1 = 'Dù khá đắt nhưng tôi vẫn đồng ý.'
sent2 = 'Mình không nuôi chúng nó được.'
sent3 = 'Nhà tôi ở trên tầng năm.'

print(CRF_POS_tagger(sent1))
print(CRF_POS_tagger(sent2))
print(CRF_POS_tagger(sent3))

Dù/C khá/R đắt/A nhưng/C tôi/P vẫn/R đồng_ý/V ./CH 
Mình/P không/R nuôi/V chúng_nó/V được/R ./CH 
Nhà_tôi/Np ở/E trên/E tầng/N năm/N ./CH 
